In [ ]:
! pip install 

In [1]:
args.NUM_CLASSES = 10
args.BATCH_SIZE = 32
args.EPOCHS = 30
args.AUGMENT = True
args.MODEL = 'pointnet'
args.LEARNING_RATE = 0.001
args.DROPOUT_RATE = 0.3
args.PRINT = True

SyntaxError: invalid syntax (<ipython-input-1-1b18610b05d0>, line 1)

In [ ]:
data_file_name = './data/parsed_data_' + str(args.NUM_POINTS) + '.pkl'
# data_file_name = './data/parsed_data.pkl'
train_points = None
test_points = None
train_labels = None
test_labels = None
class_map = None

if not os.path.isfile(data_file_name):
    data.save_data(args.NUM_POINTS)

with open(data_file_name, 'rb') as infile:
    train_points, test_points, train_labels, test_labels, class_map = pkl.load(infile)
    print('train_points:', train_points.shape)
    print('test_points:', test_points.shape)
    print('train_labels:', train_labels.shape)
    print('test_labels:', test_labels.shape)

train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))


if args.AUGMENT == True:
    train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(args.BATCH_SIZE)
else:
    train_dataset = train_dataset.shuffle(len(train_points)).batch(args.BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(args.BATCH_SIZE)

time_stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
output_dir = './outputs/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
log_dir = output_dir + args.MODEL + '_' + time_stamp + '/'
os.mkdir(log_dir)
print('MODEL LOGGGER:', log_dir)

# Construct Model
model_structure = model.model_build(NUM_POINTS=args.NUM_POINTS, NUM_CLASSES=args.NUM_CLASSES,
                                    DROPOUT_RATE=args.DROPOUT_RATE, PRINT=args.PRINT)
network = model_structure.load(MODEL=args.MODEL, log_dir=log_dir)

network.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=args.LEARNING_RATE),
    metrics=["sparse_categorical_accuracy"],
)

model_callbacks = [
    keras.callbacks.TensorBoard(log_dir=log_dir),
    #keras.callbacks.ModelCheckpoint(filepath=args.MODEL+'_{epoch:02d}.h5'),
    keras.callbacks.CSVLogger(filename=log_dir+'model_training.log')
]

network.fit(train_dataset, epochs=args.EPOCHS, validation_data=test_dataset,
            callbacks=model_callbacks)
